In [1]:
import gym
from gym import spaces
import numpy as np
from nasbench import api

2022-07-20 14:30:04.489400: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-20 14:30:04.489446: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
dataset = api.NASBench("/scratch2/sem22hs2/nasbench_full.tfrecord")

Loading dataset from file... This may take a few minutes...
Loaded dataset in 198 seconds


In [5]:
def objective_function(adjacency_mat,labeling, budget=108):
    labeling = ['input'] + list(labeling) + ['output']
    model_spec = api.ModelSpec(adjacency_mat, labeling)
    try:
        data = dataset.query(model_spec, epochs=budget)
    except api.OutOfDomainError:
        # self.record_invalid(adjacency_mat, labeling, 1, 1, 0)
        return 0, 0

    # self.record_valid(adjacency_mat, labeling, data, model_spec)
    return data["validation_accuracy"], data["training_time"]

In [6]:
# Todo: sample architecture
# Todo: Check architecture validity. Neg reward if not.

class NasBench(gym.Env):
    metadata = {"render_modes": [], "render_fps": 1}
    def __init__(self, v=7, e=9, ops=['conv1x1-bn-relu', 'conv3x3-bn-relu', 'maxpool3x3'], step_max=1000):
        assert render_mode is None # or render_mode in self.metadata["render_modes"]
        # Environment definition
        self.max_edges = e
        self.vertices = v
        self.ops = ops

        # Current state
        self.adjacency_mat = np.zeros([v,v])
        self.labeling = (v-2)*[ops[0]] # Initialize op for all layers that are not input or output layer
        
        # Helper
        self.idx_upper = np.triu_indices(v) # Indices of upper triangular matrix

        self.num_step = 0
        self.step_max =  step_max
        num_indecies_triu = len(self.idx_upper[0])
        self.observation_space = spaces.Dict(
            {
                "adjacency_mat": spaces.MultiBinary(num_indecies_triu),
                "labels": spaces.MultiDiscrete(np.array((v-2)*[len(ops)])),
            }
        )

        
    def step(self, action):
        e=self.max_edges
        v=self.vertices
        n = (v*(v+1)/2) # Number of indices in upper triag. part of matrix
        if action < n:
            # Todo: Check this changes matrix at right place
            iu = self.idx_upper
            self.adjacency_mat[iu[0][action],iu[1][action]] = not self.adjacency_mat[iu[0][action],iu[1][action]]
        else:
            o=len(self.ops)
            action = action - n
            [label_row, op] = np.unravel_index(action,[v, o])
            self.labeling[label_row] = op

        y, c = objective_function(self.adjacency_mat, self.labeling)
        reward = y
        if self.step == self.step_max:
            done = 1
        else: done = 0

        observation = None
        info = None
        return observation, reward, done, info

    def reset(self):
        self.adjacency_mat[self.idx_upper] = np.random.randint(0,2,len(self.idx_upper))
        self.labeling = np.random.randint(0,3,len(self.labeling))

In [7]:
import ray
from ray.tune.integration.wandb import WandbLoggerCallback

In [10]:
SELECT_ENV = NasBench
N_ITER = 20


ray.init()
ray.tune.run(
    "PPO",
    stop={"training_iteration": 15},
    config={
        "env": SELECT_ENV,
        "record_env": True,
        "framework": "torch",
        "num_cpus_per_worker": 2,
        "num_gpus": 2,
        "num_workers": 4,
    },
    #local_dir="logs",
    #callbacks=[WandbLoggerCallback(api_key="c36c598399c6c7f2f0b446aac164da6c7956a263", project="NasBenchV1")],
)


trainer.evaluate()

2022-07-20 14:38:48,645	WARNING util.py:214 -- The `start_trial` operation took 34.883 s, which may be a performance bottleneck.
(PPOTrainer pid=26754) 2022-07-20 14:39:16,770	ERROR worker.py:451 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::PPOTrainer.__init__() (pid=26754, ip=129.132.4.157, repr=PPOTrainer)
(PPOTrainer pid=26754)   File "/home/sem22h2/.conda/envs/RL/lib/python3.9/site-packages/ray/rllib/agents/trainer.py", line 806, in __init__
(PPOTrainer pid=26754)     self._env_id: Optional[str] = self._register_if_needed(
(PPOTrainer pid=26754)   File "/home/sem22h2/.conda/envs/RL/lib/python3.9/site-packages/ray/rllib/agents/trainer.py", line 2775, in _register_if_needed
(PPOTrainer pid=26754)     register_env(name, lambda cfg: env_object(cfg))
(PPOTrainer pid=26754)   File "/home/sem22h2/.conda/envs/RL/lib/python3.9/site-packages/ray/tune/registry.py", line 112, in register_env
(PPOTrainer pid=26754)     _global_regist

Trial name,status,loc
PPO_NasBench_d22f3_00000,RUNNING,


2022-07-20 14:39:56,423	ERROR trial_runner.py:886 -- Trial PPO_NasBench_d22f3_00000: Error processing event.
NoneType: None


Result for PPO_NasBench_d22f3_00000:
  trial_id: d22f3_00000
  


Trial name,status,loc
PPO_NasBench_d22f3_00000,ERROR,
Trial name,# failures,error file
PPO_NasBench_d22f3_00000,1,/home/sem22h2/ray_results/PPO/PPO_NasBench_d22f3_00000_0_2022-07-20_14-38-13/error.txt


2022-07-20 14:40:57,402	ERROR ray_trial_executor.py:107 -- An exception occurred when trying to stop the Ray actor:Traceback (most recent call last):
  File "/home/sem22h2/.conda/envs/RL/lib/python3.9/site-packages/ray/tune/ray_trial_executor.py", line 98, in post_stop_cleanup
    ray.get(future, timeout=0)
  File "/home/sem22h2/.conda/envs/RL/lib/python3.9/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/home/sem22h2/.conda/envs/RL/lib/python3.9/site-packages/ray/worker.py", line 1833, in get
    raise value
ray.exceptions.RayActorError: The actor died unexpectedly before finishing this task.
	class_name: PPOTrainer
	actor_id: 60dc2e994d94fc3326bbd68b01000000
	pid: 26754
	namespace: de10da65-4fd9-4358-961a-f529a1711c1c
	ip: 129.132.4.157
The actor is dead because its worker process has died. Worker exit type: CREATION_TASK_ERROR



TuneError: ('Trials did not complete', [PPO_NasBench_d22f3_00000])

In [9]:
ray.shutdown()

In [12]:
b = NasBench(v=3)

In [14]:
b.step(1)

(None, 0, 0, None)

In [15]:
b.reset()

ValueError: shape mismatch: value array of shape (2,) could not be broadcast to indexing result of shape (6,)

In [16]:
np.triu_indices(7)

(array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3,
        4, 4, 4, 5, 5, 6]),
 array([0, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 2, 3, 4, 5, 6, 3, 4, 5, 6,
        4, 5, 6, 5, 6, 6]))